$\bf{Problem\ Statement:}$ Write a Python function that takes the name of a star and returns a galpy Orbit object (setup with the star's position, distance, proper motion, and radial velocity). Apply to: GJ 440, Lacaille 8760, Vega. Check that you are finding the right star.

In [2]:
from galpy.potential import MWPotential2014
from galpy.orbit import Orbit
from gaia_tools import query
from astropy.coordinates import SkyCoord
from astroquery.simbad import Simbad

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


Write a function that takes the name of star and return position and velocity from SIMBAD

In [18]:
customSimbad = Simbad()
customSimbad.add_votable_fields('plx', 'pmra','pmdec', 'rv_value')


['main_id', 'coordinates', 'plx', 'pmra', 'pmdec', 'rv_value']

Write the SQL query statement

In [2]:
#sql = "SELECT * FROM gaiadr2.gaia_source WHERE ra BETWEEN"
#out = query.query()

Create galpy orbit, given a star's position, distance, proper motion, and radial velocity

In [ ]:
def orbit_from_name(name):
    """
    Take a star's name and return a galpy Orbit object.
    
    Args:
        name (string): the name of given star
        
    Returns:
        o (Orbit): a galpy orbit object that represents the star's orbit
    """
    # get the position and velocity in ICRS form from Simbad; search by name
    ra, dec, parallax, pm_ra, pm_dec, vr = simbad_coord(name)
    # since these coordinates are from J2000, use Gaia catalogue to find its new coordinate in current epoch
    ra, dec = correct_for_current_epoch(ra, dec, parallax, pm_ra, pm_dec, vr = None)
    # query Gaia catalogue for the star's coordinate adn velocity; the gaia's data has a 'g' prefix
    g_ra, g_dec, g_parallax, g_pm_ra, g_pm_dec, g_vr = gaia_coord(ra, dec)
    
    # if gaia does not contain the star, use simbad's result; but otherwise, prefer gaia's result
    if g_ra is None:
        data = (ra, dec, parallax, pm_ra, pm_dec, vr)
    else:
        data = (g_ra, g_dec, g_parallax, g_pm_ra, g_pm_dec, g_vr)
    
    # take care of the case where no radial velocity is given in Gaia
    data = check_for_missing_vr(data, vr)
    # convert the coordinate to galactocentric
    R,vR,vT,z,vz,phi = convert_to_galactocentric(data)
    # create the galpy object and return the result
    o = Orbit(vxvv = [R,vR,vT,z,vz,phi])
    return o